## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

### 本次作业基于大模型辅助完成的作业题已在idea处或者code处进行声明

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![image](fig/4-1.jpg)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

证明题1:
![](fig/4-2.1.jpg)

idea 2：
问题2-2：
1. 删除图中所有权重大于b的边
2. 检查剩余的图是否仍然连通
3. 如果连通，则存在最大权重不超过b的瓶颈生成树；否则不存在

In [4]:
# add your code here
def solution(graph, b):
   
    filtered_graph = {}
    for node in graph:
        filtered_graph[node] = []
        for neighbor, weight in graph[node]:
            if weight <= b:
                filtered_graph[node].append((neighbor, weight))
    
    visited = set()
    
    def dfs(node):
        visited.add(node)
        for neighbor, _ in filtered_graph[node]:
            if neighbor not in visited:
                dfs(neighbor)
    if filtered_graph:
        start_node = next(iter(filtered_graph))
        dfs(start_node)
    
    # 如果所有节点都被访问，则图连通
    return len(visited) == len(filtered_graph)

graph = {
        0: [(1, 1), (3, 4)],
        1: [(0, 1), (2, 2)],
        2: [(1, 2), (3, 3)],
        3: [(2, 3), (0, 4)]
    }
    
print(f"b=2时存在BST: {solution(graph, 2)}")  
print(f"b=3时存在BST: {solution(graph, 3)}")  

# algorithm of the liear time complexity ：时间复杂度：O(V+E)，其中V是节点数，E是边数

b=2时存在BST: False
b=3时存在BST: True


idea 3:问题2-3
1. 使用桶排序对边按权重排序，时间复杂度O(E+W)，其中W是权重范围
2. 使用并查集实现Kruskal算法

对图中所有边的权重进行排序，然后使用二分查找找到最小的权重值b，使得移除所有权重大于b的边后，图仍然连通。



In [6]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x]) 
        return self.parent[x]
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return False
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_y] = root_x
            self.rank[root_x] += 1
        return True

def findbst(graph):
    edges = []
    nodes = set()
    max_weight = 0
    
    for u in graph:
        nodes.add(u)
        for v, weight in graph[u]:
            nodes.add(v)
            if u < v:  # 避免重复添加无向边
                edges.append((u, v, weight))
                max_weight = max(max_weight, weight)
    
    # 桶排序
    buckets = [[] for _ in range(max_weight + 1)]
    for u, v, weight in edges:
        buckets[weight].append((u, v, weight))
    
    sorted_edges = []
    for bucket in buckets:
        sorted_edges.extend(bucket)
    
    # 构建节点到索引的映射
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    
    # Kruskal算法
    disjoint_set = DisjointSet(len(nodes))
    mst_edges = []
    
    for u, v, weight in sorted_edges:
        u_idx = node_to_idx[u]
        v_idx = node_to_idx[v]
        
        if disjoint_set.union(u_idx, v_idx):
            mst_edges.append((u, v, weight))
            
            # 当MST有n-1条边时停止
            if len(mst_edges) == len(nodes) - 1:
                break
    
    return mst_edges
graph = {
        0: [(1, 1), (3, 4)],
        1: [(0, 1), (2, 2)],
        2: [(1, 2), (3, 3)],
        3: [(2, 3), (0, 4)]
    }
    
bst = findbst(graph)
print(f"瓶颈生成树: {bst}")  
print(f"最大边权重: {max(weight for _, _, weight in bst)}")
# 时间复杂度：O(V+E+W)，其中V是节点数，E是边数，W是最大权重。

瓶颈生成树: [(0, 1, 1), (1, 2, 2), (2, 3, 3)]
最大边权重: 3


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [1]:
def find(V, E, l, E_prime, l_prime):
    n = len(V)
    dist = [[float('inf')] * n for _ in range(n)]

    for i in range(n):
        dist[i][i] = 0
    for (u, v) in E:
        dist[u][v] = l[(u, v)]
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    max_reduction = 0
    best_road = None
    best_city_pair = None
    
    for (u, v) in E_prime:
        road_length = l_prime[(u, v)]
        
        for s in range(n):
            for t in range(n):
                if s != t:
                    original_dist = dist[s][t]
                    new_dist = min(original_dist, dist[s][u] + road_length + dist[v][t])
                    reduction = original_dist - new_dist
                    if reduction > max_reduction:
                        max_reduction = reduction
                        best_road = (u, v)
                        best_city_pair = (s, t)
    
    return best_road, best_city_pair, max_reduction

## 正确性
1. 首先，我们使用 Floyd-Warshall 算法计算出原始图中所有点对之间的最短路径。
2. 然后，对于每条潜在的新道路 (u,v)，我们计算添加此道路后对每对城市 (s,t) 的影响：
- 如果通过新道路的路径 (s→u→v→t) 比原来的最短路径更短，则可以减少距离
- 记录导致最大减少的新道路和城市对

## 复杂度
1. 时间复杂度：O(|V|³ + |E'| × |V|²)
- Floyd-Warshall 算法：O(|V|³)
- 检查每条潜在新道路对每对城市的影响：O(|E'| × |V|²)
2. 空间复杂度：O(|V|²)，用于存储距离矩阵

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea 1:问题（1）
要将具有顶点和边容量的网络 G = (V, E) 转换为只有边容量的网络 G' = (V', E')，可以采用以下转换方法：

对于 G 中的每个顶点 v，在 G' 中创建两个顶点：v_in 和 v_out
对于每个顶点 v，添加一条从 v_in 到 v_out 的边，容量等于顶点 v 的容量
对于 G 中的每条从 u 到 v 的边，在 G' 中添加一条从 u_out 到 v_in 的边，容量与原边相同
这种转换的正确性在于：

原网络中通过顶点的流量现在受到 v_in 到 v_out 边的容量限制
顶点容量约束被转换为边容量约束
两个网络的最大流保持不变
新网络的规模：

V' = 2|V| = O(V)
E' = |E| + |V| = O(E)（因为在连通图中通常 E ≥ V-1）

#### 此题为大模型解决
idea 2：问题（2）：
1. 构建二分图 G = (L ∪ R, E)：
- 左侧 L：包含给定的 m 个顶点 (x₁, y₁), (x₂, y₂), ..., (xₘ, yₘ)
- 右侧 R：包含边界上的顶点
- 如果从 L 中的顶点可以到达 R 中的顶点（存在不与其他点路径相交的路径），则添加一条边
2. 在 G 上构建网络流模型：
- 添加源点 s，连接到 L 中所有顶点，容量为 1
- 添加汇点 t，从 R 中所有顶点连接到 t，容量为 1
- 所有二分图中的边容量设为 1
3. 在这个网络上求最大流：
- 可以使用 Hopcroft-Karp 算法（专为二分图最大匹配设计）
- 如果最大流等于 m，则存在逃离解决方案；否则不存在

时间复杂度分析：
- 构建二分图需要确定可能的路径，可以用 BFS 在 O(n²) 时间内找到从一个顶点到边界的路径
- 对 m 个顶点重复此过程：O(m·n²)
- Hopcroft-Karp 算法的时间复杂度为 O(E√V)，在我们的网络中 V = O(n²), E = O(m·n)
- 因此总的时间复杂度为 O(m·n²·√n²) = O(m·n³)


